In [6]:
import requests
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 문자 -> 인덱스 변환 함수
def char_to_index(char, char_to_idx):
    return char_to_idx[char]

# 문자열 -> 원-핫 인코딩 변환 함수
def string_to_one_hot(string, char_to_idx):
    tensor = torch.zeros(len(string), len(char_to_idx))
    for idx, char in enumerate(string):
        tensor[idx][char_to_index(char, char_to_idx)] = 1
    return tensor.to(device)

# 데이터 수집
response = requests.get("https://loripsum.net/api?plaintext&paragraphs=1000")
data = response.text
sentences = data.split("\n")

# 문자 집합 생성
char_set = sorted(list(set(data)))
char_to_idx = {char: idx for idx, char in enumerate(char_set)}

# 패딩 작업을 수행하는 함수 (수정된 부분)
def pad_sequences(sequences, max_length, padding_value):
    padded_sequences = []
    for sequence in sequences:
        sequence = np.array(sequence) # 리스트를 Numpy 배열로 변환
        padding_needed = max_length - sequence.shape[0]
        if padding_needed > 0:
            if sequence.ndim == 1:
                sequence = np.hstack((sequence, np.full(padding_needed, padding_value)))
            else:
                sequence = np.vstack((sequence, np.full((padding_needed, sequence.shape[1]), padding_value)))
        else:
            sequence = sequence[:max_length]
        padded_sequences.append(sequence)
    return padded_sequences

# 데이터셋 생성 (수정된 부분)
class ToyDataset(Dataset):
    def __init__(self, sentences, char_to_idx, max_length):
        self.sentences = sentences
        self.char_to_idx = char_to_idx
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        input_data = string_to_one_hot(sentence[:-1], self.char_to_idx).cpu().numpy() # 원-핫 인코딩된 데이터를 Numpy 배열로 변환
        target_data = [char_to_index(char, self.char_to_idx) for char in sentence[1:]]

        # 패딩 작업 수행
        input_data = pad_sequences([input_data], self.max_length, [0] * len(self.char_to_idx))[0] # 패딩 값으로 0 벡터를 사용
        target_data = pad_sequences([target_data], self.max_length, 0)[0] # -100은 패딩 값입니다. 원하는 값으로 변경하세요.

        input_data = torch.tensor(input_data, dtype=torch.float).to(device)
        target_data = torch.tensor(target_data, dtype=torch.long).to(device)

        return input_data, target_data

# 문장의 최대 길이 설정
max_length = max([len(sentence) for sentence in sentences])

# 데이터셋 및 DataLoader 생성
train_dataset = ToyDataset(sentences, char_to_idx, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True) # 배치 크기를 원하는 값으로 변경하세요.

In [7]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [8]:
def glorot_init(shape):
        init_range = np.sqrt(6.0 / (shape[0] + shape[1]))
        return torch.tensor(np.random.uniform(-init_range, init_range, size=shape), device=device, dtype=torch.float32, requires_grad=True)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.W_ih = nn.Parameter(glorot_init((input_size, hidden_size)))
        self.W_hh = nn.Parameter(glorot_init((hidden_size, hidden_size)))
        self.W_ho = nn.Parameter(glorot_init((hidden_size, output_size)))
        self.b_h = nn.Parameter(torch.zeros(hidden_size, device=device, dtype=torch.float32, requires_grad=True))
        self.b_o = nn.Parameter(torch.zeros(output_size, device=device, dtype=torch.float32, requires_grad=True))

    def forward(self, input, hidden):
        # 입력과 은닉 상태의 가중치를 곱하고 편향을 더한 후 활성화 함수를 적용합니다.
        hidden = torch.tanh(torch.matmul(input, self.W_ih) + torch.matmul(hidden, self.W_hh) + self.b_h)
        # 은닉 상태와 출력 가중치를 곱하고 편향을 더한 후 활성화 함수를 적용합니다.
        output = torch.matmul(hidden, self.W_ho) + self.b_o
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size).to(device)

In [9]:
# 모델, 손실 함수, 옵티마이저 설정
input_size = len(char_set)
hidden_size = 128
output_size = len(char_set)
model = RNN(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 학습
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        hidden = model.init_hidden(inputs.size(0))
        optimizer.zero_grad()
        loss = 0

        for i in range(inputs.size(1)):
            output, hidden = model(inputs[:, i], hidden)
            loss += criterion(output, targets[:, i])

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item() / inputs.size(1)}')

Epoch [10/100], Loss: 3.7878290857832138
Epoch [20/100], Loss: 3.266110432212186
Epoch [30/100], Loss: 2.9512082550213194
Epoch [40/100], Loss: 2.5967439004291966
Epoch [50/100], Loss: 2.558258878478456
Epoch [60/100], Loss: 2.416263052976324
Epoch [70/100], Loss: 2.331434486156306
Epoch [80/100], Loss: 2.2706783164553412
Epoch [90/100], Loss: 1.6612804145744502
Epoch [100/100], Loss: 2.1722979725370286


In [10]:
def predict(input_str, model, char_to_idx, idx_to_char, num_chars_to_generate, temperature=1.0):
    model.eval()
    input_sequence = string_to_one_hot(input_str, char_to_idx)
    hidden = model.init_hidden(1)

    for i in range(input_sequence.size(0)):
        output, hidden = model(input_sequence[i].unsqueeze(0), hidden)

    predicted_chars = []
    for _ in range(num_chars_to_generate):
        output_dist = output.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = idx_to_char[top_i.item()]
        predicted_chars.append(predicted_char)

        input_sequence = string_to_one_hot(predicted_char, char_to_idx)
        output, hidden = model(input_sequence.unsqueeze(0), hidden)

    return input_str + "".join(predicted_chars)

# 예측할 문자 개수 설정
num_chars_to_generate = 100

# 시작 문자열 설정
input_str = "Philo"

# 인덱스를 문자로 변환하는 사전 생성
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# 예측 결과 출력
predicted_text = predict(input_str, model, char_to_idx, idx_to_char, num_chars_to_generate)
print(predicted_text)









Philoushae.l




P





Rh
N





a
i




i


nf





bc


MQd











n
/
t
<



Tp

/Q


x

E



,p

